This notebook fits a linear model to the dataset to interpolate the missing time steps. The dataset we fit things to has time steps 0, 12, 24, 36, 48, and 60. The other datasets have various time steps as described in the README file. 

In [1]:
import numpy as np
import pandas as pd
import os
from glob import glob
from sklearn.linear_model import LinearRegression
import pandas as pd
from tqdm import tqdm

import tensorflow as tf
from tensorflow import keras

import numpy as np
import pandas as pd

from keras.layers import Input
from keras.layers import Dense
from keras.layers import Conv1D
from keras.layers import Conv1DTranspose
from keras.layers import Flatten, Reshape

In [2]:
##CHeck GPU
#%tensorflow_version 2.x
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [3]:
data_path = r"C:\Users\Fin Amin\Desktop\StemCellResearch\Fin_preProcessed\datasets\exp1"
dp1 = r"C:\Users\Fin Amin\Desktop\StemCellResearch\Fin_preProcessed\datasets\exp1"
dp2 = r"C:\Users\Fin Amin\Desktop\StemCellResearch\Fin_preProcessed\datasets\exp2"
dp3 = r"C:\Users\Fin Amin\Desktop\StemCellResearch\Fin_preProcessed\datasets\exp3"
dp4 = r"C:\Users\Fin Amin\Desktop\StemCellResearch\Fin_preProcessed\datasets\exp4"
dp5 = r"C:\Users\Fin Amin\Desktop\StemCellResearch\Fin_preProcessed\datasets\exp5"
dp6 = r"C:\Users\Fin Amin\Desktop\StemCellResearch\Fin_preProcessed\datasets\exp6"
allExp = [dp1, dp2, dp3, dp4, dp5, dp6]

In [4]:
data_path_head = r"C:\Users\Fin Amin\Desktop\StemCellResearch\Fin_preProcessed\datasets"
matrix_path = "regulator_gene_matrix.csv"
templatePath = r"C:\Users\Fin Amin\Desktop\StemCellResearch\Fin_preProcessed\datasets\template.csv"

In [5]:
def getCSVs(data_path_head):
    PATH = data_path_head
    EXT = "*.csv"
    all_csv_files = [file
                     for path, subdir, files in os.walk(PATH)
                     for file in glob(os.path.join(path, EXT))]
    return all_csv_files


In [6]:
exps = []
for p in allExp:
    exps.append(getCSVs(p))

In [7]:
pd.read_csv(exps[1][0])

,Unnamed: 0,AT1G04530.1_T36_1,AT1G08680.5_S196_1,AT1G08820.5_S130_2,AT1G10940.1_S357_1,AT1G11310.3_S503_1,AT1G11480.2_S343_1,AT1G12830.1_S46_1,AT1G13020.1_S436_1,AT1G13980.2_S961_1,...,AT5G54900.1_S332_1,AT5G56030.1_S288_1,AT5G56340.1_S150_2,AT5G56340.1_S151_2,AT5G56980.1_S120_1,AT5G57870.1_S487_1,AT5G58140.4_S30_1,AT5G64740.1_S136_1,AT5G67200.1_S371_1,ATCG00490.1_S208_1
0,0,0.254943,-1.000000,0.663585,0.440312,-1,0.635664,0.734571,-1.000000,0.312044,...,-1.000000,0.400712,-1,0.523033,-1,0.561590,0.403879,0.642049,-1.000000,0.443478
1,10,0.338509,-1.000000,0.612395,0.655007,-1,-1.000000,0.590564,-1.000000,0.353872,...,-1.000000,0.445519,-1,0.586067,-1,0.541903,0.426403,0.495559,0.550973,0.419991
2,20,-1.000000,0.544463,0.596307,0.596367,-1,0.629406,0.579586,0.726602,-1.000000,...,0.509166,-1.000000,-1,0.642262,-1,0.586480,0.569283,-1.000000,0.596188,-1.000000
3,40,-1.000000,0.528085,-1.000000,0.523557,-1,0.588043,0.580612,-1.000000,0.354997,...,0.573489,0.392001,-1,0.602732,-1,0.544163,0.423905,-1.000000,0.578650,-1.000000


In [8]:
allExperiments = os.listdir(data_path_head)
print(allExperiments)

['exp1', 'exp2', 'exp3', 'exp4', 'exp5', 'exp6', 'readme.txt', 'template.csv']


In [9]:
def read_files():
    '''
    *Changed*
    currently hardcoded for only one file. 
    change code a bit for reading multiple files.
    '''
    replicates = []
    for f in allExperiments:
        
        #genes_intensities_data_matrix = pd.read_csv(file_path_intensities, index_col = 0)
        #print(os.listdir(data_path))
        replicate_files = os.listdir(data_path)
        print('replicate files:',replicate_files)
        # i = 0
        for file in replicate_files:

            #print('file name:',file)
            #print('value of i:',i)
            genes_intensities_data_matrix = pd.read_csv(os.path.join(data_path, file), index_col = 0, on_bad_lines='skip')
            #print('genes_intensities_data_matrix:',  genes_intensities_data_matrix.head())
            replicates.append(genes_intensities_data_matrix.values)
            # i+=1
        
    genes_intensities_data_matrix = genes_intensities_data_matrix.values
    regulator_gene_matrix = np.loadtxt(matrix_path)
    
    return np.asarray(replicates), regulator_gene_matrix.astype(np.float32)

In [10]:
_, regulator_gene_matrix = read_files()

replicate files: ['Col0-tot3_Col_rep_A_Normalized.csv', 'Col0-tot3_Col_rep_B_Normalized.csv', 'Col0-tot3_Col_rep_C_Normalized.csv', 'Col0-tot3_Col_rep_D_Normalized.csv']
replicate files: ['Col0-tot3_Col_rep_A_Normalized.csv', 'Col0-tot3_Col_rep_B_Normalized.csv', 'Col0-tot3_Col_rep_C_Normalized.csv', 'Col0-tot3_Col_rep_D_Normalized.csv']
replicate files: ['Col0-tot3_Col_rep_A_Normalized.csv', 'Col0-tot3_Col_rep_B_Normalized.csv', 'Col0-tot3_Col_rep_C_Normalized.csv', 'Col0-tot3_Col_rep_D_Normalized.csv']
replicate files: ['Col0-tot3_Col_rep_A_Normalized.csv', 'Col0-tot3_Col_rep_B_Normalized.csv', 'Col0-tot3_Col_rep_C_Normalized.csv', 'Col0-tot3_Col_rep_D_Normalized.csv']
replicate files: ['Col0-tot3_Col_rep_A_Normalized.csv', 'Col0-tot3_Col_rep_B_Normalized.csv', 'Col0-tot3_Col_rep_C_Normalized.csv', 'Col0-tot3_Col_rep_D_Normalized.csv']
replicate files: ['Col0-tot3_Col_rep_A_Normalized.csv', 'Col0-tot3_Col_rep_B_Normalized.csv', 'Col0-tot3_Col_rep_C_Normalized.csv', 'Col0-tot3_Col_rep

In [11]:
# datasetTotal = []
# for i in all_csv_files:
#     datasetTotal.append(pd.read_csv(i))
# print(len(datasetTotal)) 

In [12]:
###the first 4 csvs are the ones we are trying to match

In [13]:
pd_exps = []
for e in exps:
    temp = []
    for sample in e:
        temp.append(pd.read_csv(sample))
            
    pd_exps.append(pd.concat(temp))    
        
len(pd_exps)        

6

In [14]:
lr = []

i = 0
for d in tqdm(pd_exps):
    y = d.iloc[:, 1:] 
    x = d.iloc[:, 0]
    x = x.to_numpy()
    x = np.reshape(x, (-1, 1))
    print(y.shape)
    reg = LinearRegression().fit(x, y)
    lr.append(reg)

100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 92.28it/s]

(24, 245)
(16, 245)
(25, 245)
(25, 245)
(16, 245)
(16, 245)


In [15]:
tmp = pd.read_csv(templatePath)
newData = []
time = np.array([0, 12, 24, 36, 48, 60])
time = np.reshape(time, (-1, 1) )
for r in lr:
    df = tmp.copy()
    guess = r.predict(time)
    df.iloc[:, 1:] = guess
    newData.append(df)

In [16]:
syn = r"C:\Users\Fin Amin\Desktop\StemCellResearch\Fin_preProcessed\synData"
for i in range(len(newData)):
    p = "syn" + str(i) + ".csv"
    #p = os.path.join(syn,p)
    newData[i].to_csv(p)

In [17]:
#enc_dec = model(regulator_gene_matrix, 245, 6, 245)
len(regulator_gene_matrix)

741

In [18]:
'''Can I just append -1 to non existant time steps?'''

'Can I just append -1 to non existant time steps?'

In [19]:
exps[3][0]

'C:\\Users\\Fin Amin\\Desktop\\StemCellResearch\\Fin_preProcessed\\datasets\\exp4\\Dorrit_HT_rep_A_Normalized.csv'

In [52]:
'''
For Dorrit: Just add 15 min
For Jelle: Add 30 min and 50 min
For Nighttime: Add 60 min and 540 min
'''

'\nFor Dorrit: Just add 15 min\nFor Jelle: Add 30 min and 50 min\nFor Nighttime: Add 60 min and 540 min\n'

In [60]:
pd_Dorrit = []
i = 1
tdor = np.array([15.0])
tdor = np.reshape(tdor, (-1, 1) )
for e in exps:
    if i == 3 or i == 4: #dorrit is exp 3 and 4
        temp = []
        for sample in e:
            trainingSample = pd.read_csv(sample)
            trainingSample = trainingSample.to_numpy()
    
            out = np.array(lr[i-1].predict(tdor))
            out = np.hstack((np.array([[15.0]]),out))

            trainingSample = np.vstack((trainingSample, out))
            
            temp.append(trainingSample)

        pd_Dorrit.append(np.concatenate(temp))
        i = i + 1
    else:
        i = i + 1
        continue
        
#print(len(pd_Dorrit)) 

#0, 10, 20, 40 minutes 4 total 
pd_jelle = []
i = 1
tjell = np.array([30.0,50.0])
tjell = np.reshape(tjell, (-1, 1) )

for e in exps:
    if i == 2: #jelle is exp 2
        temp = []
        for sample in e:
            trainingSample = pd.read_csv(sample)
            trainingSample = trainingSample.to_numpy()
    
            out = np.array(lr[i-1].predict(tjell))
            out30 = out[0]
            out30 = np.hstack((np.array([30.0]),out30))
            out50 = out[1]
            out50 = np.hstack((np.array([50.0]),out50))
            trainingSample = np.insert(trainingSample, 3, out30, axis=0)
            trainingSample = np.vstack((trainingSample, out50))
            temp.append(trainingSample)

        pd_jelle.append(np.concatenate(temp))
        i = i + 1
    else:
        i = i + 1
        continue
        
#print(len(pd_jelle)) 

#Nighttimecourse: 15, 30, 120, and 960 minutes 4 total
#Add 60 min and 540 min
pd_night = []
i = 1
tnight = np.array([60.0,540.0])
tnight = np.reshape(tnight, (-1, 1) )

for e in exps:
    if i == 5 or i == 6: #night time is exp 5 and 6
        temp = []
        for sample in e:
            trainingSample = pd.read_csv(sample)
            trainingSample = trainingSample.to_numpy()
    
            out = np.array(lr[i-1].predict(tnight))
            out60 = out[0]
            out60 = np.hstack((np.array([60.0]),out60))
            out540 = out[1]
            out540 = np.hstack((np.array([540.0]),out540))
            trainingSample = np.insert(trainingSample, 2, out60, axis=0)
            trainingSample = np.vstack((trainingSample, out540))
            temp.append(trainingSample)

        pd_night.append(np.concatenate(temp))
        i = i + 1
    else:
        i = i + 1
        continue
        


In [58]:
pd.DataFrame(pd_Dorrit[0]).head(6)

,0,1,2,3,4,5,6,7,8,9,...,236,237,238,239,240,241,242,243,244,245
0,0.0,0.383464,0.715858,-1.000000,0.480260,0.578379,0.586765,-1.000000,-1.000000,0.405191,...,0.783118,0.403324,-1.000000,0.507345,-1.000000,-1.000000,-1.000000,-1.000000,0.516111,0.487210
1,3.0,0.315177,0.662139,0.494593,-1.000000,-1.000000,-1.000000,-1.000000,0.194132,0.536485,...,0.790711,-1.000000,-1.000000,0.550469,0.635320,-1.000000,0.535349,-1.000000,0.511575,-1.000000
2,6.0,-1.000000,0.662936,0.522142,-1.000000,-1.000000,0.476192,0.659048,-1.000000,0.503296,...,0.783947,-1.000000,-1.000000,0.542648,0.544033,0.331349,0.553913,-1.000000,0.532768,0.550551
3,9.0,-1.000000,0.683119,-1.000000,0.657177,-1.000000,-1.000000,-1.000000,-1.000000,0.478903,...,0.761514,-1.000000,-1.000000,0.496945,0.440845,0.330121,0.589792,-1.000000,0.535499,0.600833
4,12.0,-1.000000,0.642001,0.516811,0.566484,-1.000000,0.633065,0.648447,-1.000000,0.508585,...,0.754793,-1.000000,0.447949,0.493508,0.533393,-1.000000,0.552273,-1.000000,0.551298,0.581482
5,15.0,-1.255162,0.326625,0.358605,0.394652,0.081532,-0.516276,0.410669,-0.474162,0.318071,...,0.740802,-0.663686,-0.447179,0.546686,0.166530,-0.459497,0.975579,-0.344807,0.554497,0.699229


In [59]:
pd.DataFrame(pd_jelle[0]).head(6)

,0,1,2,3,4,5,6,7,8,9,...,236,237,238,239,240,241,242,243,244,245
0,0.0,0.254943,-1.000000,0.663585,0.440312,-1.000000,0.635664,0.734571,-1.000000,0.312044,...,-1.000000,0.400712,-1.000000,0.523033,-1.000000,0.561590,0.403879,0.642049,-1.000000,0.443478
1,10.0,0.338509,-1.000000,0.612395,0.655007,-1.000000,-1.000000,0.590564,-1.000000,0.353872,...,-1.000000,0.445519,-1.000000,0.586067,-1.000000,0.541903,0.426403,0.495559,0.550973,0.419991
2,20.0,-1.000000,0.544463,0.596307,0.596367,-1.000000,0.629406,0.579586,0.726602,-1.000000,...,0.509166,-1.000000,-1.000000,0.642262,-1.000000,0.586480,0.569283,-1.000000,0.596188,-1.000000
3,30.0,-0.329754,-0.454583,-0.828393,0.582746,-0.689224,0.178233,0.309607,-0.297329,-0.461788,...,-0.024981,-0.697509,-0.153204,0.546957,0.161868,0.291320,-0.109625,-0.444279,0.007077,-0.254379
4,40.0,-1.000000,0.528085,-1.000000,0.523557,-1.000000,0.588043,0.580612,-1.000000,0.354997,...,0.573489,0.392001,-1.000000,0.602732,-1.000000,0.544163,0.423905,-1.000000,0.578650,-1.000000
5,50.0,-0.248367,-0.040930,-1.194954,0.605865,-0.952363,0.098652,0.307698,-0.336617,-0.278098,...,0.290480,-0.934986,-0.339247,0.595122,0.154406,0.091483,-0.428788,-0.819855,0.223532,-0.254089


In [61]:
pd.DataFrame(pd_night[0]).head(6)

,0,1,2,3,4,5,6,7,8,9,...,236,237,238,239,240,241,242,243,244,245
0,15.0,0.621149,0.248149,0.646971,0.535597,-1.000000,-1.000000,0.512092,-1.000000,0.426530,...,0.416995,0.412596,0.364535,0.364535,-1.000000,-1.000000,0.543389,-1.000000,-1.000000,-1.000000
1,30.0,0.655506,0.656264,0.667739,0.461199,-1.000000,-1.000000,0.455976,-1.000000,-1.000000,...,-1.000000,0.392651,-1.000000,-1.000000,-1.000000,0.554882,0.551343,0.383940,0.380069,-1.000000
2,60.0,0.581643,0.035233,0.413407,0.356722,-0.886713,-0.883456,-0.113644,-0.078039,-0.205803,...,-0.120083,0.384624,0.015083,0.015083,-0.499256,-0.030345,0.274032,-0.506228,-0.705887,-0.311236
3,120.0,0.659016,0.624873,0.417764,0.502807,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,...,-1.000000,0.404282,0.327027,0.327027,-1.000000,0.541116,0.520808,0.523931,-1.000000,-1.000000
4,960.0,0.632129,-1.000000,-1.000000,0.352709,-1.000000,-1.000000,0.555723,-1.000000,0.369151,...,0.559370,0.380280,0.310967,0.310967,-1.000000,0.619718,0.479290,-1.000000,0.469834,0.412569
5,540.0,0.648406,-0.309206,0.062314,0.402868,-0.951535,-0.953027,-0.226377,-0.064201,0.090384,...,-0.032600,0.199006,0.190407,0.190407,-0.577672,0.263170,0.378858,-0.775655,-0.293291,0.085321


In [65]:
def saveLisa():
    
    i = 0
    for x in pd_Dorrit:
        p = "dorrit_syn" + str(i) + ".csv"
        pd.DataFrame(x).to_csv(p)
        i = i + 1
        
    i = 0
    for x in pd_jelle:
        p = "jelle_syn" + str(i) + ".csv"
        pd.DataFrame(x).to_csv(p)
        i = i + 1
        
    i = 0
    for x in pd_night:
        p = "night_syn" + str(i) + ".csv"
        pd.DataFrame(x).to_csv(p)
        i = i + 1
        


In [66]:
saveLisa()